In [ ]:
#import SciServer
#from SciServer import Authentication, LoginPortal, Config, CasJobs, SkyQuery, SciDrive, SkyServer
from SciServer import CasJobs
#import os
#import sys
#import json
#from io import StringIO
#from io import BytesIO
#from PIL import Image
#import matplotlib.pyplot as plt
print('Imported SciServer modules')

In [ ]:
import pandas
import numpy as np
from pprint import pprint
from datetime import datetime, timedelta
print('Imported other needed modules')

In [ ]:
# Read the help document for the entire SciServer package
help(SciServer)

In [ ]:
# Read the help document for the CasJobs module
help(CasJobs)

In [ ]:
# Define some very simple test cases for the purposes of this example notebook

CasJobs_TestDatabase = "MyDB" # use your MyDB - normally you won't need to change this
CasJobs_TestQuery = "select 4 as Column1, 5 as Column2 " # toy query, simply returns two columns of constants: 4, 5.
CasJobs_TestTableName1 = "MyNewtable1"    # the name of a new table to be created (doesn't yet exist)
CasJobs_TestTableName2 = "MyNewtable37"    # the name of a new table to be created (doesn't yet exist)
CasJobs_TestTableCSV = u"Column1,Column2\n6,7\n"    # toy CSV file: 1 row x 2 columns of constants: 6, 7.
CasJobs_TestCSVFile = "SciScriptTestFile.csv"       # a more real CSV file. It must be included in this dir in current container.
CasJobs_TestFitsFile = "SciScriptTestFile.fits"     # a FITS file to upload

print('Test case variables set.')

In [ ]:
#get user schema info

casJobsId = CasJobs.getSchemaName()
print(casJobsId)

In [ ]:
# Returns the following information abaout the tables in your MyDB (as a Python dictionary object):
### Size: size of the table (in kB)
### Name: the name of the table
### Rows: the number of rows the table contains
### Date: the date of the table's creation, as the number of 10-microsecond intervals elapsed 1 AD

# These convenience functions will print your list of tables in human-readable format

def better_createdate(cjCreateDate):     # returns table create date as a datetime object
    createsec = cjCreateDate / 10000000  # Divide by 10 million to get seconds elapsed since 1 AD
    firstday = datetime(1, 1, 1, 0, 0)   # Save 1 AD as "firstday"
    created = firstday + timedelta(seconds=createsec)  # Get calendar date on which table was created
    return (created)
def tables_formatted(tables):   # better formatted printing of a tables dictionary (output of get_tables)
    for thistable in tables:
        print('Table name: ',thistable['Name'])
        print('Rows: ',thistable['Rows'])
        print('Size (kB): ',thistable['Size'])
        betterdate = better_createdate(thistable['Date'])
        print('Created time: ',betterdate.strftime('%Y-%m-%d %H:%M:%S'))
        print('\n')

tables = CasJobs.getTables(context=CasJobs_TestDatabase)
print('Tables in '+CasJobs_TestDatabase+':\n')
tables_formatted(tables)

In [ ]:
#executes a quick SQL query and store results in a pandas dataframe:


df = CasJobs.executeQuery(sql=CasJobs_TestQuery, context=CasJobs_TestDatabase, format="pandas")
df

# Other options for return format (format=...):
# 'json': a JSON string containing the query results
# 'dict': a dictionary created from the JSON string containing the query results
# 'csv': a csv string
# 'readable': an object of type io.StringIO, which has the .read() method and wraps a csv string that can be passed into pandas.read_csv for example
# 'StringIO': an object of type io.StringIO, which has the .read() method and wraps a csv string that can be passed into pandas.read_csv for example
# 'fits': an object of type io.BytesIO, which has the .read() method and wraps the result in fits format
# 'BytesIO': an object of type io.BytesIO, which has the .read() method and wraps the result in fits format

# CasJobs.executeQuery(sql, context, format)

In [ ]:
#submit a job, which inserts the query results into a table in the MyDB database context. 
#Wait until the job is done and get its status.

def translate_status(status):
    if (status == 0):
        status_word = 'Ready'
    elif (status == 1):
        status_word = 'Started'
    elif (status == 2):
        status_word = 'Cancelling'
    elif (status == 3):
        status_word = 'Cancelled'
    elif (status == 4):
        status_word = 'Failed'
    elif (status == 5):
        status_word = 'Finished'
    else:
        status_word = 'Status not found!!!!!!!!!'
    return (status_word)

def jobDescriber(jobDescription):
    print('JobID: ',jobDescription['JobID'])
    print('Status: ',jobDescription['Status'],' (',translate_status(jobDescription['Status']),')')
    print('Message: ',jobDescription['Message'])
    print('Created_Table: ',jobDescription['Created_Table'])
    print('Rows: ',jobDescription['Rows'])
    wait = pandas.to_datetime(jobDescription['TimeStart']) - pandas.to_datetime(jobDescription['TimeSubmit'])
    duration = pandas.to_datetime(jobDescription['TimeEnd']) - pandas.to_datetime(jobDescription['TimeStart'])
    print('Wait time: ',wait.seconds,' seconds')
    print('Query duration: ',duration.seconds, 'seconds')

    
thequery = CasJobs_TestQuery + 'into MyDB.' + CasJobs_TestTableName2

jobId = CasJobs.submitJob(sql=thequery, context="MyDB")
print('Submitting query:\n',thequery)
print('\n')
print('Job submitted with jobId = ',jobId)
print('\n')
jobDescription = CasJobs.waitForJob(jobId=jobId, verbose=True)
print('\n')
print('Information about the job:')
jobDescriber(jobDescription)

In [ ]:
bigtablename = 'hugetable'

verylongquery = 'select top 150000 *\n'
verylongquery += 'into ' + CasJobs_TestDatabase + '.'+ bigtablename + '\n'
verylongquery += 'from photoobjall'

#thequery = CasJobs_TestQuery + 'into MyDB.' + CasJobs_TestTableName2
thequery = verylongquery

print('Submitting query:\n',thequery)
print('\n')

jobId = CasJobs.submitJob(sql=thequery, context="DR14")

print('Job submitted with jobId = ',jobId)
print('\n')

jobDescription = CasJobs.waitForJob(jobId=jobId, verbose=True)
print('\n')

print('Information about the job:')
jobDescriber(jobDescription)
#pprint(jobDescription)